In [203]:
import tensorflow as tf
import tensorflow_hub as hub

In [204]:
import pandas as pd
import numpy as np

In [206]:
train_df = pd.read_csv("Disaster tweet/train.csv")
test_df = pd.read_csv("Disaster tweet/test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


import random
random_int = random.randint(0,len(df))
random_int

for i in range(5):
    random_int = random.randint(0,len(train_df))
    print(f"Target: {train_df['target'][random_int]}" , "(real disaster)" if train_df['target'][random_int]>0 else "(not real disaster)")
    print(f"Text:\n{df['text'][random_int]}")
    print('----\n')


In [207]:
train_df_shuffled = train_df.sample(frac=1,random_state=42)

Splitting our data into test and train

In [208]:
from sklearn.model_selection import train_test_split
train_sen,val_sen,train_labels,val_labels = train_test_split(train_df_shuffled['text'].to_numpy(),
                                                             train_df_shuffled['target'].to_numpy(),
                                                             random_state=42,
                                                            test_size=.1)

In [209]:
len(train_sen),len(val_sen)

(6851, 762)

In [210]:
#viewing first 5 training samples
train_sen[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

#writing a function that will turn our text data into numbers. for more info: 
https://www.tensorflow.org/api_docs/python/tf/keras/layers/TextVectorization

In [211]:
#average words per tweet
avg_word = round(sum([len(i.split()) for i in train_sen])/len(train_sen))
avg_word

15

In [212]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
text_vectorizer = TextVectorization(max_tokens=1000,
                                   output_mode='int',
                                   output_sequence_length=avg_word)
text_vectorizer

In [213]:
text_vectorizer.adapt(train_sen)

In [214]:
#before converting any text from train_sen lets convert any text to numbers using text_vectorizer layer
text_to_number = text_vectorizer(["I am going to convert this sentence to numbers"])
text_to_number

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  8, 160, 104,   5,   1,  19,   1,   5,   1,   0,   0,   0,   0,
          0,   0]], dtype=int64)>

In [215]:
random_text = random.choice(train_sen)
print(f"Original text:\n\n {random_text}")
print("----")
print(f"Vectorized text:")
text_vectorizer([random_text])

Original text:

 Pretty teen Hayden Ryan poses and strips out of her purple top http://t.co/qew4c5M1xd View and download video
----
Vectorized text:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[766,   1,   1,   1,   1,   7,   1,  36,   6,  81,   1, 212,   1,
        976,   7]], dtype=int64)>

In [216]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5] # most common tokens (notice the [UNK] token for "unknown" words)
bottom_5_words = words_in_vocab[-5:] # least common tokens
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}") 
print(f"Bottom 5 least common words: {bottom_5_words}")

Number of words in vocab: 1000
Top 5 most common words: ['', '[UNK]', 'the', 'a', 'in']
Bottom 5 least common words: ['reported', 'r', 'pray', 'playlist', 'patience']


Lets write a function to convert these numbers to feature vectors. For more info: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding

In [217]:
from tensorflow.keras.layers import Embedding

In [218]:
embedding_layer = Embedding(input_dim=1000,
                           output_dim=128,
                           input_length=avg_word,
                           name='embedding_1')
embedding_layer

Converting our 'text_to_number' variable to feature vector

In [219]:
embedded_text = embedding_layer(text_to_number)
embedded_text

<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.00643777,  0.00809912, -0.0333337 , ..., -0.0014866 ,
         -0.02233075, -0.00463134],
        [ 0.04601005,  0.02017364, -0.01703423, ...,  0.04098071,
          0.02005376, -0.00837458],
        [-0.00627888,  0.03613004, -0.03284682, ...,  0.04677888,
         -0.0294435 , -0.04086154],
        ...,
        [-0.01204009,  0.01908055, -0.0382763 , ..., -0.03221116,
         -0.02657125,  0.02125778],
        [-0.01204009,  0.01908055, -0.0382763 , ..., -0.03221116,
         -0.02657125,  0.02125778],
        [-0.01204009,  0.01908055, -0.0382763 , ..., -0.03221116,
         -0.02657125,  0.02125778]]], dtype=float32)>

We will now train the model using a pre trained model from tensorflow hub and fit that model to our data

In [589]:
pretrained_model = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder-large/5",
                                 trainable=False,
                                 input_shape=[],
                                 dtype='string')

In [591]:
pre_trained_2 = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3",
                                 trainable=False,
                                 input_shape=[],
                                 dtype='string')

OSError: SavedModel file does not exist at: C:\Users\prabh\AppData\Local\Temp\tfhub_modules\c9fe785512ca4a1b179831acb18a0c6bfba603dd\{saved_model.pbtxt|saved_model.pb}

In [221]:
pretrained_model

In [222]:
model_1 = tf.keras.Sequential([
    pretrained_model,
#     text_vectorizer,
#     embedding_layer,
    tf.keras.layers.Dense(64,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
    
])

model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [223]:
model_1_history = model_1.fit(train_sen,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sen, val_labels),
                              )

Epoch 1/5
215/215 [==============================] - 234s 534ms/step - loss: 0.5035 - accuracy: 0.7885 - val_loss: 0.4268 - val_accuracy: 0.8215
Epoch 2/5
215/215 [==============================] - 101s 472ms/step - loss: 0.4003 - accuracy: 0.8269 - val_loss: 0.4176 - val_accuracy: 0.8228
Epoch 3/5
215/215 [==============================] - 90s 420ms/step - loss: 0.3830 - accuracy: 0.8326 - val_loss: 0.4168 - val_accuracy: 0.8255
Epoch 4/5
215/215 [==============================] - 90s 418ms/step - loss: 0.3723 - accuracy: 0.8375 - val_loss: 0.4140 - val_accuracy: 0.8228
Epoch 5/5
215/215 [==============================] - 89s 415ms/step - loss: 0.3641 - accuracy: 0.8427 - val_loss: 0.4141 - val_accuracy: 0.8241


In [226]:
print(tf.__version__)

2.5.0


In [227]:
from platform import python_version
print(python_version())

3.8.8


In [228]:
model_1.save("model_1_disaster_tweet_local")

INFO:tensorflow:Assets written to: model_1_disaster_tweet_local\assets


INFO:tensorflow:Assets written to: model_1_disaster_tweet_local\assets


In [229]:
loaded_model_1 = tf.keras.models.load_model('model_1_disaster_tweet_local')
loaded_model_1

In [230]:
model_1.evaluate(val_sen,val_labels)

24/24 [==============================] - 10s 395ms/step - loss: 0.4141 - accuracy: 0.8241


[0.41410452127456665, 0.8241469860076904]

In [231]:
loaded_model_1.evaluate(val_sen,val_labels)

24/24 [==============================] - 25s 403ms/step - loss: 0.4141 - accuracy: 0.8241


[0.41410452127456665, 0.8241469860076904]

In [232]:
pred_prob = loaded_model_1.predict(val_sen)
pred_prob[:10]

array([[0.24357572],
       [0.7291944 ],
       [0.9931166 ],
       [0.12791267],
       [0.71145445],
       [0.71302086],
       [0.9891956 ],
       [0.9541272 ],
       [0.97506726],
       [0.1465087 ]], dtype=float32)

In [233]:
pred_labels = tf.squeeze(tf.round(pred_prob)).numpy()
pred_labels[:10]

array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)

In [290]:
def predict_on_sen(model,sentence):
    pred_prob = model.predict([sentence])
    pred_label = tf.squeeze(tf.round(pred_prob)).numpy()
    print(f"Pred: {pred_label}", "(real disaster)" if pred_label > 0 else "(not real disaster)",f"Prob: {pred_prob[0][0]}")
    print(f"Text:\n{sentence}")

In [292]:
sentence_1 = 'flood and Dead bodies everywhere..Good God bless these souls'
predict_on_sen(model=loaded_model_1,sentence=sentence_1)

Pred: 1.0 (real disaster) Prob: 0.5109326839447021
Text:
flood and Dead bodies everywhere..Good God bless these souls


In [332]:
wrong_pred_df = pd.DataFrame({
    'Text': val_sen,
    'Target':val_labels,
    'Pred_label':pred_labels,
    'Pred_prob':tf.squeeze(pred_prob)
})

wrong_pred_df[:10]

,Text,Target,Pred_label,Pred_prob
0,DFR EP016 Monthly Meltdown - On Dnbheaven 2015...,0,0.0,0.243576
1,FedEx no longer to transport bioterror germs i...,0,1.0,0.729194
2,Gunmen kill four in El Salvador bus attack: Su...,1,1.0,0.993117
3,@camilacabello97 Internally and externally scr...,1,0.0,0.127913
4,Radiation emergency #preparedness starts with ...,1,1.0,0.711454
5,Investigators rule catastrophic structural fai...,1,1.0,0.713021
6,How the West was burned: Thousands of wildfire...,1,1.0,0.989196
7,Map: Typhoon Soudelor's predicted path as it a...,1,1.0,0.954127
8,Ûª93 blasts accused Yeda Yakub dies in Karach...,1,1.0,0.975067
9,My ears are bleeding https://t.co/k5KnNwugwT,0,0.0,0.146509


In [304]:
most_wrong = wrong_pred_df[wrong_pred_df['Target']!=wrong_pred_df['Pred_label']].sort_values(by='Pred_prob',ascending=False)

In [525]:
most_wrong[:10]

,Text,Target,Pred_label,Pred_prob
619,Emergency Response and Hazardous Chemical Mana...,0,1.0,0.918610
31,? High Skies - Burning Buildings ? http://t.co...,0,1.0,0.917468
49,@madonnamking RSPCA site multiple 7 story high...,0,1.0,0.877314
11,The Five Fatal Flaws in the Iran Deal https://...,0,1.0,0.869626
303,Trafford Centre film fans angry after Odeon ci...,0,1.0,0.864984
344,Air Group is here to the rescue! We have 24/7 ...,0,1.0,0.822100
759,FedEx will no longer transport bioterror patho...,0,1.0,0.789781
718,Haley Lu Richardson Fights for Water in The La...,0,1.0,0.788961
317,Photo: postapocalypticflimflam: Prodding aroun...,0,1.0,0.784829
628,@noah_anyname That's where the concentration c...,0,1.0,0.770379


In [315]:
len(train_df) == len(train_sen)+len(val_sen)

True

In [514]:
#finding the most wrong predicted samples in training dataset
# false positive
for row in most_wrong[:20].itertuples():
    _,text,_,_,_ = row
    print(text)
    x = train_df['id'][train_df['text']==text]
    print(x,'\n','----','\n')

Emergency Response and Hazardous Chemical Management: Principles and Practices http://t.co/4sSuyhkgRB http://t.co/TDerBtgZ2k
1527    2209.0
Name: id, dtype: float64 
 ---- 

? High Skies - Burning Buildings ? http://t.co/uVq41i3Kx2 #nowplaying
1206    1737.0
Name: id, dtype: float64 
 ---- 

@madonnamking RSPCA site multiple 7 story high rise buildings next to low density character residential in an area that floods
3991    5670.0
Name: id, dtype: float64 
 ---- 

The Five Fatal Flaws in the Iran Deal https://t.co/ztfEAd8GId via @YouTube
3594    5134.0
Name: id, dtype: float64 
 ---- 

Trafford Centre film fans angry after Odeon cinema evacuated following false fire alarm   http://t.co/6GLDwx71DA
3343    4787.0
Name: id, dtype: float64 
 ---- 

Air Group is here to the rescue! We have 24/7 Emergency Service! Learn more about it here - http://t.co/9lyx7zMtHE http://t.co/5PbC96rTMJ
3226    4630.0
Name: id, dtype: float64 
 ---- 

FedEx will no longer transport bioterror pathogens in wake

In [517]:
# false negative
for row in most_wrong[-20:].itertuples():
    _,text,_,_,_ = row
    print(text)
    x = train_df['id'][train_df['text']==text]
    print(x,'\n','----','\n')

@willienelson We need help! Horses will die!Please RT &amp; sign petition!Take a stand &amp; be a voice for them! #gilbert23 https://t.co/e8dl1lNCVu
257    365.0
Name: id, dtype: float64 
 ---- 

New post from @darkreading http://t.co/8eIJDXApnp New SMB Relay Attack Steals User Credentials Over Internet
461    665.0
Name: id, dtype: float64 
 ---- 

If I fall is men GOD @Praiz8 is d bomb well av always known dat since 2008 bigger u I pray sir
1080    1560.0
Name: id, dtype: float64 
 ---- 

I get to smoke my shit in peace
6221    8880.0
Name: id, dtype: float64 
 ---- 

@reriellechan HE WAS THE LICH KING'S FIRST CASUALTY BLOCK ME BACK I HATE YOU! http://t.co/0Gidg9U45J
1454    2097.0
Name: id, dtype: float64 
 ---- 

Sadly before she could save humanity Ursula drowned in the drool of a protoshoggoth but at least she sort of died doing what she loved.
2923    4202.0
Name: id, dtype: float64 
 ---- 

Just came back from camping and returned with a new song which gets recorded tomorrow. C

In [512]:
top_10_false_positive_id=[]
for row in most_wrong[:10].itertuples():
    _,text,_,_,_ = row
#     print(text)
    x = train_df['id'][train_df['text']==text]
#     print(x)
    for i in x:
        top_10_false_positive_id.append(i)
print(top_10_false_positive_id)

[2209.0, 1737.0, 5670.0, 5134.0, 4787.0, 4630.0, 836.0, 9439.0, 8339.0, 6862.0]


In [513]:
top_10_false_negative_id=[]
for row in most_wrong[-10:].itertuples():
    _,text,_,_,_ = row
#     print(text)
    x = train_df['id'][train_df['text']==text]
#     print(x)
    for i in x:
        top_10_false_negative_id.append(i)
print(top_10_false_negative_id)

[7954.0, 9315.0, 519.0, 2675.0, 8786.0, 3625.0, 9759.0, 2782.0, 3221.0, 3369.0]


Lets manually correct the dataset and reimport it and re train it on same train and test data and see the results

In [499]:
top_10_false_positive_index=[]
for row in most_wrong[:10].itertuples():
    _,text,_,_,_ = row
    x = train_df[train_df['text']==text]
    for i in x.itertuples():
        index,_,_,_,_,_ = i
        top_10_false_positive_index.append(index)
print(top_10_false_positive_index)

[1527, 1206, 3991, 3594, 3343, 3226, 578, 6592, 5836, 4821]


In [526]:
top_20_false_negative_index=[]
for row in most_wrong[-20:].itertuples():
    _,text,_,_,_ = row
    x = train_df[train_df['text']==text]
    for i in x.itertuples():
        index,_,_,_,_,_ = i
        top_20_false_negative_index.append(index)
print(top_20_false_negative_index)

[257, 461, 1080, 6221, 1454, 2923, 2511, 4154, 6239, 5596, 5574, 6514, 361, 1861, 6160, 2522, 6812, 1934, 2250, 2342]


In [527]:
corrected_df = train_df

In [528]:
for i in top_20_false_negative_index:
    corrected_df.at[i,'target']=0

In [534]:
corrected_df.to_csv('C:\\Users\\prabh\\Videos\\Disaster tweet\\corrected disaster tweet train.csv')

In [535]:
train_df.iloc[257]

id                                                      365.0
keyword                                          annihilation
location                                                  NaN
text        @willienelson We need help! Horses will die!Pl...
target                                                    0.0
Name: 257, dtype: object

In [536]:
test_df

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [549]:
ids=[]
target=[]
for row in test_df.itertuples():
    _,id,_,_,text=row
    ids.append(id)
#     print(text)
    pred_prob = loaded_model_1.predict([text])
    pred_label = tf.squeeze(tf.round(pred_prob)).numpy()
    target.append(pred_label)

In [551]:
submission_df = pd.DataFrame({
    'id':ids,
    'target':target
})

submission_df.head()

,id,target
0,0,1.0
1,2,1.0
2,3,1.0
3,9,1.0
4,11,1.0


In [553]:
len(submission_df),len(test_df)

(3263, 3263)

In [556]:
test_sentences = test_df["text"].to_list()
test_samples = random.sample(test_sentences, 10)

In [557]:
test_samples

['RT- Udhampur terror attack: Militants attack police post 2 SPOs injured: Suspected militants tonight at... http://t.co/wN414hp9hZ #News',
 "I'm an emotional wreck someone hold me until they upload the damn video #whatstheimportantvideo",
 "@ReasonVsFear 'One?'  Oh nonononono.  This hijacking's been going on forever but I've seen it about a dozen diff places today alone.",
 '@hitchBOT I heard you might rise from the ashes of Armageddon. .go hitchbot ..like the true superstar you are.',
 '11-Year-Old Boy Charged With Manslaughter of Toddler: Report: An 11-year-old boy has been charged with manslaughter over the fatal sh...',
 'Rape victim dies as she sets herself ablaze: A 16-year-old girl died of burn injuries as she set herself ablaze\x89Û_ http://t.co/UK8hNrbOob',
 '#Business People are finally panicking about cable TV http://t.co/FcfSSlxMy2 http://t.co/9xFzkpdmQz',
 '@al_thegoon i like hurricane habor next to slix flags but they got like 8 rides but they wavy &amp; hershey park',
 

In [560]:
import re
import string

def remove_URL(text):
    url = re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"", text)

# https://stackoverflow.com/questions/34293875/how-to-remove-punctuation-marks-from-a-string-in-python-3-x-using-translate/34294022
def remove_punct(text):
    translator = str.maketrans("", "", string.punctuation)
    return text.translate(translator)

string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [561]:
pattern = re.compile(r"https?://(\S+|www)\.\S+")
for t in df.text:
    matches = pattern.findall(t)
    for match in matches:
        print(t)
        print(match)
        print(pattern.sub(r"", t))
    if len(matches) > 0:
        break

@bbcmtd Wholesale Markets ablaze http://t.co/lHYXEOHY6C
t
@bbcmtd Wholesale Markets ablaze 


In [562]:
df = pd.read_csv("Disaster tweet/train.csv")

In [563]:
df["text"] = df.text.map(remove_URL) # map(lambda x: remove_URL(x))
df["text"] = df.text.map(remove_punct)

In [564]:
# remove stopwords
# pip install nltk
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Stop Words: A stop word is a commonly used word (such as “the”, “a”, “an”, “in”) that a search engine
# has been programmed to ignore, both when indexing entries for searching and when retrieving them 
# as the result of a search query.
stop = set(stopwords.words("english"))

# https://stackoverflow.com/questions/5486337/how-to-remove-stop-words-using-nltk-or-python
def remove_stopwords(text):
    filtered_words = [word.lower() for word in text.split() if word.lower() not in stop]
    return " ".join(filtered_words)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\prabh\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [565]:
df["text"] = df.text.map(remove_stopwords)

In [569]:
# from sklearn.metrics import train_test_split

train_sen_filter,val_sen_filter,train_labels,val_labels = train_test_split(df['text'],
                                                                          df['target'],
                                                                          random_state=42,
                                                                          test_size=.1)

In [571]:
len(train_sen_filter),len(val_sen_filter)

(6851, 762)

In [572]:
model_2 = tf.keras.Sequential([
    pretrained_model,
    tf.keras.layers.Dense(64,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
    
])

model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [574]:
model_2_history = model_2.fit(train_sen_filter,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sen_filter, val_labels),
                              )

Epoch 1/5
215/215 [==============================] - 163s 639ms/step - loss: 0.5065 - accuracy: 0.7889 - val_loss: 0.4178 - val_accuracy: 0.8189
Epoch 2/5
215/215 [==============================] - 112s 520ms/step - loss: 0.4139 - accuracy: 0.8219 - val_loss: 0.3998 - val_accuracy: 0.8268
Epoch 3/5
215/215 [==============================] - 99s 461ms/step - loss: 0.3981 - accuracy: 0.8251 - val_loss: 0.3965 - val_accuracy: 0.8320
Epoch 4/5
215/215 [==============================] - 100s 466ms/step - loss: 0.3871 - accuracy: 0.8335 - val_loss: 0.3989 - val_accuracy: 0.8255
Epoch 5/5
215/215 [==============================] - 99s 461ms/step - loss: 0.3796 - accuracy: 0.8348 - val_loss: 0.3965 - val_accuracy: 0.8255


In [579]:
model_2.evaluate(val_sen_filter,val_labels),model_1.evaluate(val_sen_filter,val_labels)

24/24 [==============================] - 7s 266ms/step - loss: 0.3704 - accuracy: 0.8346


([0.39649683237075806, 0.8254593014717102],
 [0.37043341994285583, 0.834645688533783])

In [576]:
sentence_1 = "Is this a disaster tweet"
sentence_2 = 'flood and Dead bodies everywhere..Good God bless these souls'
sentence_3 = "Life is so fickle and unpredictable. I cannot process the passing of this great of our sport and also a person I got to know off the field. RIP #goat. Greatest to turn the cricket ball."
sentence_4 = "NSW floods live updates: Evacuation warning as Manly Dam spills, Sydney roads closed for dangerous flash flooding - ABC News "

In [577]:
for i in [sentence_1,sentence_2,sentence_3,sentence_4]:
  predict_on_sen(model_2,i)
  print('\n')

Pred: 0.0 (not real disaster) Prob: 0.24924486875534058
Text:
Is this a disaster tweet


Pred: 1.0 (real disaster) Prob: 0.5465414524078369
Text:
flood and Dead bodies everywhere..Good God bless these souls


Pred: 0.0 (not real disaster) Prob: 0.248977392911911
Text:
Life is so fickle and unpredictable. I cannot process the passing of this great of our sport and also a person I got to know off the field. RIP #goat. Greatest to turn the cricket ball.


Pred: 1.0 (real disaster) Prob: 0.9923743009567261
Text:
NSW floods live updates: Evacuation warning as Manly Dam spills, Sydney roads closed for dangerous flash flooding - ABC News 




In [586]:
pre_trained_2 = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3",
                                 trainable=False,
                                 input_shape=[],
                                 dtype='string')

FileNotFoundError: Op type not registered 'SentencepieceOp' in binary running on PRABHANSHU. Make sure the Op and Kernel are registered in the binary running in this process. Note that if you are loading a saved graph which used ops from tf.contrib, accessing (e.g.) `tf.contrib.resampler` should be done before importing the graph, as contrib ops are lazily registered when the module is first accessed.
 If trying to load on a different device from the computational device, consider using setting the `experimental_io_device` option on tf.saved_model.LoadOptions to the io_device such as '/job:localhost'.

In [ ]:
model_3 = tf.keras.Sequential([
    pre_trained_2,
    tf.keras.layers.Dense(64,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
    
])

model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
model_3_history = model_3.fit(train_sen_filter,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sen_filter, val_labels),
                              )

In [ ]:
model_3.evaluate(val_sen_filter,val_labels)

In [ ]:
for i in [sentence_1,sentence_2,sentence_3,sentence_4]:
  predict_on_sen(model_3,i)
  print('\n')